In [ ]:
# Hipotesis No 1 a desarrollar en el ejercicio por Alvin Luperon
#Para los estudiantes inscritos en el curso virtual AAA, se busca determinar si con el historial de combinación de compromiso conductual (clics totales, días activos, promedio de clics por día, score_avg, entre otras)
# y características demográficas (puntuación promedio, créditos, género, región, nivel educativo, nivel socioeconómico y grupo de edad) de los años académicos 2013J y 2014J del dataset de OULAD
# puede ser utilizada para:
# Primero: Predecir por estudiante el estado de aprobado/reprobado en el año académico 2025J con un modelo de Regresión Logística.
# Segundo: Estimar su puntuación final del curso en el año académico 2025J con un modelo de Random Forest.

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, ConfusionMatrixDisplay,
    roc_curve, auc, precision_recall_curve,
    classification_report, mean_absolute_error, mean_squared_error, r2_score
)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from xgboost import XGBRegressor

In [ ]:
# Configuración de los dataset, limpieza de dataset, manejo valores null o missing, uniones, manejo de variables categóricas y codificación ordinal

# Rutas base de carpetas
ruta = "./oulad"
rutaEdaImg = "./edaimg"
rutaDataPred = "./datatopredict"
rutaOrdMapping = "./ordmapping"
rutaResult_Pred = "./result_pred"
rutaMetrics_models = "./metrics_models"

# === 1. Cargar datos ===
student_info = pd.read_csv(os.path.join(ruta, "studentInfo.csv"))
student_registration = pd.read_csv(os.path.join(ruta, "studentRegistration.csv"))
student_assessments = pd.read_csv(os.path.join(ruta, "studentAssessment.csv"))
student_vle = pd.read_csv(os.path.join(ruta, "studentVle.csv"))

print("✅ Datos cargados correctamente.")

# === 2. Filtrar por módulo "AAA" ===
student_info = student_info[student_info["code_module"] == "AAA"]
student_vle = student_vle[student_vle["code_module"] == "AAA"]
student_ids = student_info["id_student"]
student_assessments = student_assessments[student_assessments["id_student"].isin(student_ids)]
student_registration = student_registration[student_registration["id_student"].isin(student_ids)]

print("✅ Datos filtrados por módulo 'AAA'.")

# === 3. Métricas de interacción con la plataforma ===
# Convertir la columna 'date' a numérica, forzando errores a NaN
student_vle["date"] = pd.to_numeric(student_vle["date"], errors="coerce")

# Calcular el total de clics por estudiante
clicks = student_vle.groupby("id_student")["sum_click"].sum().reset_index(name="total_clicks")

# Calcular el número de días activos por estudiante
active_days = student_vle.groupby("id_student")["date"].nunique().reset_index(name="active_days")

# Unir los totales de clics y días activos
clicks_summary = pd.merge(clicks, active_days, on="id_student", how="left")

# Calcular el promedio de clics por día activo, manejando divisiones por cero
clicks_summary["avg_clicks_per_day"] = clicks_summary["total_clicks"] / clicks_summary["active_days"].replace(0, pd.NA)

print("✅ Métricas de interacción con la plataforma calculadas.")

# === 4. Métricas académicas ===
# Convertir la columna 'score' a numérica, forzando errores a NaN
student_assessments["score"] = pd.to_numeric(student_assessments["score"], errors="coerce")

# Calcular el promedio de score por estudiante
avg_score = student_assessments.groupby("id_student")["score"].mean().reset_index(name="avg_score")

# Calcular el promedio y total de créditos estudiados por estudiante
credits_summary = student_info.groupby("id_student")["studied_credits"].agg(
    avg_credits="mean", total_credits="sum"
).reset_index()

print("✅ Métricas académicas calculadas.")

# === 5. Unir métricas ===
summary = clicks_summary.merge(avg_score, on="id_student", how="left")
summary = summary.merge(credits_summary, on="id_student", how="left")

# Ajustar el promedio de créditos (dividir por 12, asumiendo 12 créditos por año estándar)
summary["avg_credits"] = summary["avg_credits"] / 12

print("✅ Métricas unidas y promedios ajustados.")

# === 6. Agregar variables categóricas ===
categorical_cols = ["gender", "region", "highest_education", "imd_band", "age_band", "final_result", "code_module",
                    "code_presentation"]
student_categoricals = student_info[["id_student"] + categorical_cols].drop_duplicates()
summary = summary.merge(student_categoricals, on="id_student", how="left")

print("✅ Variables categóricas agregadas.")

# === 7. Codificación ordinal y guardado de mappings en un solo archivo ===

# Definir los mappings para cada variable categórica
gender_mapping = {"M": 0, "F": 1}
region_mapping = {v: i for i, v in enumerate(sorted(summary["region"].dropna().unique()))}
highest_education_mapping = {v: i for i, v in enumerate(sorted(summary["highest_education"].dropna().unique()))}
imd_band_mapping = {v: i for i, v in enumerate(sorted(summary["imd_band"].dropna().unique()))}
age_band_mapping = {v: i for i, v in enumerate(sorted(summary["age_band"].dropna().unique()))}

# Aplicar la codificación ordinal al DataFrame principal
summary["gender_ordinal"] = summary["gender"].map(gender_mapping)
summary["region_ordinal"] = summary["region"].map(region_mapping)
summary["highest_education_ordinal"] = summary["highest_education"].map(highest_education_mapping)
summary["imd_band_ordinal"] = summary["imd_band"].map(imd_band_mapping)
summary["age_band_ordinal"] = summary["age_band"].map(age_band_mapping)

# Crear una lista para almacenar DataFrames de cada mapeo
all_mappings_dfs = []

# Añadir cada mapeo a la lista con nombres de columna apropiados
# Género
df_gender_map = pd.DataFrame(gender_mapping.items(), columns=['valor_original', 'valor_ordinal'])
df_gender_map['variable_categorica'] = 'gender'
all_mappings_dfs.append(df_gender_map)

# Región
df_region_map = pd.DataFrame(region_mapping.items(), columns=['valor_original', 'valor_ordinal'])
df_region_map['variable_categorica'] = 'region'
all_mappings_dfs.append(df_region_map)

# Nivel educativo más alto
df_edu_map = pd.DataFrame(highest_education_mapping.items(), columns=['valor_original', 'valor_ordinal'])
df_edu_map['variable_categorica'] = 'highest_education'
all_mappings_dfs.append(df_edu_map)

# Banda IMD
df_imd_map = pd.DataFrame(imd_band_mapping.items(), columns=['valor_original', 'valor_ordinal'])
df_imd_map['variable_categorica'] = 'imd_band'
all_mappings_dfs.append(df_imd_map)

# Banda de edad
df_age_map = pd.DataFrame(age_band_mapping.items(), columns=['valor_original', 'valor_ordinal'])
df_age_map['variable_categorica'] = 'age_band'
all_mappings_dfs.append(df_age_map)

# Concatenar todos los DataFrames de mapeo en un solo DataFrame
all_ordinal_mappings_df = pd.concat(all_mappings_dfs, ignore_index=True)

# Reordenar las columnas para que 'variable_categorica' sea la primera
all_ordinal_mappings_df = all_ordinal_mappings_df[['variable_categorica', 'valor_original', 'valor_ordinal']]

# Guardar el DataFrame combinado en un único archivo CSV
all_ordinal_mappings_df.to_csv(os.path.join(rutaOrdMapping, 'all_ordinal_mappings.csv'), index=False)

print("✅ Todos los mappings de codificación ordinal guardados en 'all_ordinal_mappings.csv'.")

# === 8. Variable binaria de aprobación ===
summary["passed"] = summary["final_result"].apply(lambda x: 1 if x in ["Pass", "Distinction"] else 0)

print("✅ Variable binaria de aprobación 'passed' creada.")

# === 9. Limpieza y orden de columnas ===
# Eliminar las columnas categóricas originales, ya que ahora tenemos sus versiones ordinales
summary = summary.drop(columns=["gender", "region", "highest_education", "imd_band", "age_band", "final_result"],
                       errors="ignore")

# Reordenar las columnas para mayor claridad
cols = list(summary.columns)
if all(col in cols for col in ["id_student", "code_module", "code_presentation"]):
    cols.remove("code_module")
    cols.remove("code_presentation")
    id_index = cols.index("id_student") + 1
    cols = cols[:id_index] + ["code_module", "code_presentation"] + cols[id_index:]
    summary = summary[cols]

print("✅ Columnas limpiadas y reordenadas.")

# === 0. Eliminar filas con valores faltantes ===
summary.dropna(subset=[
    "total_clicks", "active_days", "avg_clicks_per_day", "avg_score",
    "gender_ordinal", "region_ordinal", "highest_education_ordinal",
    "imd_band_ordinal", "age_band_ordinal", "passed", "avg_credits", "total_credits",
])

# === 10. Guardar resultados ===

summary.to_csv(os.path.join(ruta, "H1 - oulad_click_summary_AAA_course.csv"), index=False)
print("✅ Archivo final guardado como 'oulad_click_summary_AAA_course.csv'.")



In [ ]:
#Cargar los dataset a utilizar y hacer analisis descriptivos de los mismos

#  Cargar los datos de entrenamiento
df_train = summary
# Cargar el dataset de predicción
df_predict = pd.read_csv(os.path.join(rutaDataPred, "datasetapredecir.csv"))

# Mostrar información general de los DataFrames de entrenamiento 
df_train.info()
df_train.describe().to_csv(os.path.join(ruta, "H1 - resumen_estadistico_oulad_click_summary_AAA_course.csv"))

# Mostrar información general de los DataFrames de predicción 
df_predict.info()
df_predict.describe().to_csv(os.path.join(rutaDataPred, "H1 - resumen_estadistico_datasetapredecir.csv"))


In [ ]:
# Cargar los datos
df_train  # Reemplaza con el nombre real de tu archivo
mappings_df = all_ordinal_mappings_df

# Crear diccionario de mapeos
mapping_dict = {}
for var in mappings_df['variable_categorica'].unique():
    sub_df = mappings_df[mappings_df['variable_categorica'] == var]
    mapping_dict[var] = dict(zip(sub_df['valor_ordinal'], sub_df['valor_original']))

# Variables categóricas y numéricas
categorical_vars = [
    'gender_ordinal', 'region_ordinal',
    'highest_education_ordinal', 'imd_band_ordinal', 'age_band_ordinal', 'passed'
]
numeric_vars = [
    'avg_score', 'total_clicks', 'active_days', 'avg_clicks_per_day', 'total_credits', 'avg_credits'
]

# Inicializar listas para almacenar resultados
anova_data = []
ttest_data = []

# Pruebas estadísticas
for cat_var in categorical_vars:
    if cat_var not in df_train.columns:
        continue
    for num_var in numeric_vars:
        if num_var not in df_train.columns:
            continue
        groups = [group[num_var].dropna() for name, group in df_train.groupby(cat_var)]
        if len(groups) > 1:
            f_stat, p_val = stats.f_oneway(*groups)
            anova_data.append({
                'Variable': num_var,
                'Agrupado_por': cat_var,
                'F-statistic': f_stat,
                'p-value': p_val
            })
        if df_train[cat_var].nunique() == 2:
            unique_vals = df_train[cat_var].dropna().unique()
            group1 = df_train[df_train[cat_var] == unique_vals[0]][num_var].dropna()
            group2 = df_train[df_train[cat_var] == unique_vals[1]][num_var].dropna()
            if len(group1) > 1 and len(group2) > 1:
                t_stat, p_val = stats.ttest_ind(group1, group2)
                ttest_data.append({
                    'Variable': num_var,
                    'Agrupado_por': cat_var,
                    'T-statistic': t_stat,
                    'p-value': p_val
                })

# Convertir resultados en DataFrames
anova_df = pd.DataFrame(anova_data)
ttest_df = pd.DataFrame(ttest_data)

# Convertir los resultados a formato horizontal (pivotar por agrupación)
anova_horizontal = anova_df.pivot(index='Variable', columns='Agrupado_por', values='p-value')
ttest_horizontal = ttest_df.pivot(index='Variable', columns='Agrupado_por', values='p-value')

# Formatear los valores p con 6 decimales sin notación científica
anova_horizontal = anova_horizontal.map(lambda x: f"{x:.6f}")
ttest_horizontal = ttest_horizontal.map(lambda x: f"{x:.6f}")

# Restablecer el índice para que 'Variable' sea una columna
anova_horizontal.reset_index(inplace=True)
ttest_horizontal.reset_index(inplace=True)

# Guardar resultados si se desea
anova_horizontal.to_csv(os.path.join(rutaResult_Pred, "H1 - TestDataSet_resultados_anova.csv"), index=False)
ttest_horizontal.to_csv(os.path.join(rutaResult_Pred, "H1 - TestDataSet_resultados_ttest.csv"), index=False)

# Mostrar los primeros resultados
print("Resultados ANOVA:")
display(anova_horizontal)

print("\nResultados T-test:")
display(ttest_horizontal)

In [ ]:
# Variables predictoras y variable objetivo
features = [
    'total_clicks', 'active_days', 'avg_clicks_per_day',
    'avg_credits', 'total_credits',
    'gender_ordinal', 'region_ordinal',
    'highest_education_ordinal', 'imd_band_ordinal', 'age_band_ordinal'
]
X = df_train[features]
y = df_train['passed']

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === 4. Validación básica ===
print("✅ Variables predictoras:", list(X.columns))
print("Distribución de la variable objetivo:\n", y.value_counts())

In [ ]:
# Hipotesis 1 - parte 1: Predecir su estado de aprobado/reprobado en el año académico 2025J con un modelo de Regresión Logística.

#Modelo de regresión logística

# Entrenar modelo de regresión logística
model = LogisticRegression(max_iter=3000)
model.fit(X_train, y_train)

# Entrenar modelo
model = LogisticRegression(max_iter=2000)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# calcular métricas de evaluación
mse = mean_squared_error(y_test, y_proba)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_proba)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# crear un DataFrame para las métricas
metrics = {
    "Metric": ["Accuracy", "Precision", "Recall", "F1 Score", "MSE", "RMSE", "R² Score"],
    "Value": [accuracy, precision, recall, f1, mse, rmse, r2]
}
df_metrics = pd.DataFrame(metrics)
# Guardar las métricas en un archivo CSV
df_metrics.to_csv(os.path.join(rutaMetrics_models, "H1 - logistic_regression_metrics.csv"), index=False)

# imprimir las métricas
print("Logistic Regression Metrics:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

# Aplicar el modelo para predecir
X_pred = df_predict[features]
df_predict['predicted_passed'] = model.predict(X_pred)

# Guardar resultados
df_predict.to_csv(os.path.join(rutaResult_Pred, "H1 - Regresión Logística pred_estudiantes pass or not.csv"),
                  index=False)
print("\nH1 - Regresión Logística pred_estudiantes.csv'")

In [ ]:
# Graficas de metricas del modelo
#estilo de gráficos
sns.set(style="whitegrid", context="talk", palette="husl")

# Calcular métricas
cm = confusion_matrix(y_test, y_pred)
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)
precision, recall, _ = precision_recall_curve(y_test, y_proba)

# Crear figura con 3 gráficos
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Matriz de confusión
disp_cm = ConfusionMatrixDisplay(confusion_matrix=cm)
disp_cm.plot(ax=axes[0], cmap="cividis", colorbar=False)
axes[0].set_title("Matriz de Confusión")

# Curva ROC
axes[1].plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
axes[1].plot([0, 1], [0, 1], 'k--')
axes[1].set_title("Curva ROC")
axes[1].set_xlabel("False Positive Rate")
axes[1].set_ylabel("True Positive Rate")
axes[1].legend(loc="lower right")

# Curva Precisión-Recall
axes[2].plot(recall, precision)
axes[2].set_title("Curva Precisión-Recall")
axes[2].set_xlabel("Recall")
axes[2].set_ylabel("Precision")

plt.tight_layout()
plt.savefig(os.path.join(rutaEdaImg, "H1 - Regresión Logística - Métricas del Modelo.png"))
plt.show()

In [ ]:
# Analisis exploratorio EDA por variables ordinales

# Cargar los datos de predicción y los mappings
df = df_predict
mappings = all_ordinal_mappings_df

# Variables ordinales
ordinal_columns = [
    'gender_ordinal', 'region_ordinal',
    'highest_education_ordinal', 'imd_band_ordinal', 'age_band_ordinal'
]

# Crear diccionarios de mapeo
mapping_dicts = {}
for col in ordinal_columns:
    var_name = col.replace('_ordinal', '')
    mapping = mappings[mappings['variable_categorica'] == var_name][['valor_ordinal', 'valor_original']]
    mapping_dicts[col] = dict(zip(mapping['valor_ordinal'], mapping['valor_original']))

# Reemplazar valores ordinales por etiquetas
for col in ordinal_columns:
    df[col] = df[col].map(mapping_dicts[col])

# Configurar estilo de gráficos
sns.set(style="whitegrid")
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Gráfico 1: Boxplot con colores verde (aprobado) y rojo (no aprobado)
palette_box = {0: "red", 1: "green"}
sns.boxplot(x='age_band_ordinal', y='total_clicks', hue='predicted_passed', data=df, ax=axes[0], palette=palette_box)
axes[0].set_title('Total Clicks por Edad y Predicción')
axes[0].set_xlabel('Grupo de Edad')
axes[0].set_ylabel('Total Clicks')

# Corregir leyenda manualmente
handles, _ = axes[0].get_legend_handles_labels()
axes[0].legend(handles=handles, labels=['No Aprobó', 'Sí Aprobó'], title='¿Aprobó?')

# Gráfico 2: Barras horizontales apiladas por nivel educativo
edu_counts = df.groupby(['highest_education_ordinal', 'predicted_passed']).size().unstack(fill_value=0)
edu_props = edu_counts.div(edu_counts.sum(axis=1), axis=0)
edu_props = edu_props.loc[edu_props.sum(axis=1).sort_values().index]

axes[1].barh(edu_props.index, edu_props[0], color='red', label='No Aprobó')
axes[1].barh(edu_props.index, edu_props[1], left=edu_props[0], color='green', label='Sí Aprobó')

for i, (rej, apr) in enumerate(zip(edu_props[0], edu_props[1])):
    axes[1].text(rej / 2, i, f'{rej:.0%}', va='center', ha='center', color='white', fontsize=9)
    axes[1].text(rej + apr / 2, i, f'{apr:.0%}', va='center', ha='center', color='white', fontsize=9)

axes[1].set_title('Proporción de Aprobados por Nivel Educativo')
axes[1].set_xlabel('Proporción')
axes[1].set_ylabel('Nivel Educativo')
axes[1].legend(title='Predicción')

# Gráfico 3: Proporción de aprobados por región y género
region_gender = df.groupby(['region_ordinal', 'gender_ordinal'])['predicted_passed'].mean().reset_index()
region_gender_pivot = region_gender.pivot(index='region_ordinal', columns='gender_ordinal', values='predicted_passed')
region_gender_pivot.plot(kind='bar', ax=axes[2], color=['#1f77b4', '#ff7f0e'])

axes[2].set_title('Proporción de Aprobados por Región y Género')
axes[2].set_xlabel('Región')
axes[2].set_ylabel('Proporción de Aprobados')
axes[2].legend(title='Género', labels=['M', 'F'], loc='lower right')
axes[2].set_xticklabels(region_gender_pivot.index, rotation=45, ha='right')

plt.tight_layout()
plt.savefig(os.path.join(rutaEdaImg, "H1 - EDA - Variables Ordinales.png"))
plt.show()


In [ ]:
# Análisis y clustering de los datos de estudiantes

# Cargar los datos
df = df_predict

# Variables numéricas para análisis y clustering
numeric_cols = ['total_clicks', 'active_days', 'avg_clicks_per_day',
                'total_credits', 'avg_credits', 'predicted_passed']
features = ['total_clicks', 'active_days', 'avg_clicks_per_day',
            'total_credits', 'avg_credits']

# Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[numeric_cols])
X_scaled_features = scaler.fit_transform(df[features])

# PCA para reducción de dimensionalidad
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

pca_3d = PCA(n_components=3)
X_pca_3d = pca_3d.fit_transform(X_scaled)

# K-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_scaled_features)
df['cluster'] = clusters

# Calcular resumen por cluster
cluster_summary = df.groupby('cluster').agg({
    'total_clicks': 'mean',
    'active_days': 'mean',
    'avg_clicks_per_day': 'mean',
    'total_credits': 'mean',
    'avg_credits': 'mean',
    'predicted_passed': lambda x: round(x.mean() * 100, 2)
}).rename(columns={'predicted_passed': 'approval_rate (%)'})

cluster_summary = cluster_summary.round(2)

# Mostrar el resumen
print("Resumen de Perfiles de Clusters:")
display(cluster_summary)

# Guardar el resumen de clusters en un archivo CSV
cluster_summary.to_csv(os.path.join(rutaResult_Pred, "H1 - Clustering resumen perfiles clusters.csv"), index=True)

# Preparar figura
fig = plt.figure(figsize=(20, 6))

# Gráfico 1: Matriz de correlación
ax1 = fig.add_subplot(1, 3, 1)
corr = df[numeric_cols].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', ax=ax1)
ax1.set_title("Matriz de Correlación")

# Gráfico 2: Clustering con PCA 2D
ax2 = fig.add_subplot(1, 3, 2)
scatter = ax2.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=clusters, cmap='Set1', alpha=0.7)
ax2.set_title("Clustering de Estudiantes (PCA 2D)")
ax2.set_xlabel("PCA 1")
ax2.set_ylabel("PCA 2")

# Gráfico 3: Dispersión 3D con PCA
ax3 = fig.add_subplot(1, 3, 3, projection='3d')
scatter3d = ax3.scatter(X_pca_3d[:, 0], X_pca_3d[:, 1], X_pca_3d[:, 2],
                        c=clusters, cmap='Set1', alpha=0.7)
ax3.set_title("Dispersión 3D por Cluster (PCA)")
ax3.set_xlabel("PCA 1")
ax3.set_ylabel("PCA 2")
ax3.set_zlabel("PCA 3")

plt.tight_layout()
plt.savefig(os.path.join(rutaEdaImg, "H1 - Clustering y Análisis de Estudiantes.png"))
plt.show()


In [ ]:
# Hipotesis 1 - parte 2: Estimar su puntuación final del curso en el año académico 2025J con un modelo de Random Forest.

In [ ]:
# Cargar los datos
historical_df = summary
future_df = pd.read_csv(os.path.join(rutaDataPred, "datasetapredecir.csv"))

In [ ]:
# Definir características y variable objetivo
features = [
    'total_clicks', 'active_days', 'avg_clicks_per_day', 'avg_credits', 'total_credits',
    'gender_ordinal', 'region_ordinal', 'highest_education_ordinal',
    'imd_band_ordinal', 'age_band_ordinal'
]
target = 'avg_score'

# Eliminar filas con valores faltantes
historical_df = historical_df.dropna(subset=features + [target])

# Eliminar outliers usando el método IQR
Q1 = historical_df[target].quantile(0.25)
Q3 = historical_df[target].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
filtered_df = historical_df[(historical_df[target] >= lower_bound) & (historical_df[target] <= upper_bound)]

# Separar variables y etiquetas
X = filtered_df[features]
y = filtered_df[target]

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo RandomForest Regressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Evaluación del modelo
y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("=== Métricas del Modelo RandomForest (sin outliers) ===")
print(f"MAE:  {mae:.2f}")
print(f"MSE:  {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.2f}")

# Clasificación binaria basada en score >= 70
y_test_class = y_test.apply(lambda x: 1 if x >= 70 else 0)
y_pred_class = pd.Series(y_pred).apply(lambda x: 1 if x >= 70 else 0)

# Métricas de clasificación
acc = accuracy_score(y_test_class, y_pred_class)
prec = precision_score(y_test_class, y_pred_class)
rec = recall_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class)

print("\n=== Métricas de Clasificación Derivadas del Score ===")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")

# crear un DataFrame para las métricas
metrics = {
    "Metric": ["Accuracy", "Precision", "Recall", "F1 Score", "MAE", "MSE", "RMSE", "R² Score"],
    "Value": [acc, prec, rec, f1, mae, mse, rmse, r2]
}
df_metrics = pd.DataFrame(metrics)
# Guardar las métricas en un archivo CSV
df_metrics.to_csv(os.path.join(rutaMetrics_models, "H1.2- random_forest_regressor_metrics.csv"), index=False)

# Aplicar modelo al dataset de 2025J
X_future = future_df[features]
future_df['predicted_score'] = rf_model.predict(X_future)


# Clasificación detallada
def clasificar(score):
    if score <= 60:
        return 'No Aprobado'
    elif 60 < score <= 70:
        return 'En Recuperación'
    elif 70 < score <= 90:
        return 'Aprobado'
    else:
        return 'Honorífico'


future_df['clasificacion'] = future_df['predicted_score'].apply(clasificar)

# Exportar dataset enriquecido
future_df.to_csv(os.path.join(rutaResult_Pred, "H1.2 - Random Forest pred_estudiantes score.csv"), index=False)

In [ ]:
#gráficos de métricas del modelo Random Forest

#estilo de gráficos
sns.set(style="whitegrid", context="talk", palette="husl")

# Matriz de confusión
cm = confusion_matrix(y_test_class, y_pred_class)

# Importancia de variables
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Crear visualizaciones
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Matriz de confusión
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Matriz de Confusión')
axes[0].set_xlabel('Predicción')
axes[0].set_ylabel('Real')

# Importancia de variables
sns.barplot(x='Importance', y='Feature', data=feature_importance_df, ax=axes[1], palette='viridis')
axes[1].set_title('Importancia de Variables')

# Score real vs predicho
axes[2].scatter(y_test, y_pred, alpha=0.6)
axes[2].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
axes[2].set_xlabel('Score Real')
axes[2].set_ylabel('Score Predicho')
axes[2].set_title('Score Real vs. Score Predicho')

plt.tight_layout()
plt.savefig(os.path.join(rutaEdaImg, "H1.2 - Random Forest - Métricas del Modelo.png"))
plt.show()


In [ ]:
#estilo de gráficos
sns.set(style="whitegrid", context="talk", palette="husl")

# Análisis de errores del modelo Random Forest
errors = y_test - y_pred
abs_errors = np.abs(errors)

# Crear visualizaciones
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Error absoluto por score real
axes[0].scatter(y_test, abs_errors, alpha=0.6, edgecolor='black', linewidth=0.5)
axes[0].set_xlabel("Score Real")
axes[0].set_ylabel("Error Absoluto")
axes[0].set_title("Error Absoluto vs. Score Real")

# 2. Histograma de errores
sns.histplot(errors, bins=20, kde=True, ax=axes[1], color='orange')
axes[1].set_title("Distribución de Errores (Score Real - Predicho)")
axes[1].set_xlabel("Error")

# 3. Curva de residuos
axes[2].scatter(y_pred, errors, alpha=0.6, edgecolor='black', linewidth=0.5)
axes[2].axhline(0, color='red', linestyle='--')
axes[2].set_xlabel("Score Predicho")
axes[2].set_ylabel("Residuo")
axes[2].set_title("Curva de Residuos")

plt.tight_layout()
plt.savefig(os.path.join(rutaEdaImg, "H1.2 - Random Forest - Análisis de Errores.png"))
plt.show()


In [ ]:

# Analisis exploratorio EDA por variables ordinales

# Estilo visual
sns.set(style="whitegrid", context="talk")

# Cargar los datos
df = future_df
mappings = all_ordinal_mappings_df

# Crear diccionarios de mapeo
region_map = mappings[mappings['variable_categorica'] == 'region'].set_index('valor_ordinal')[
    'valor_original'].to_dict()
gender_map = mappings[mappings['variable_categorica'] == 'gender'].set_index('valor_ordinal')[
    'valor_original'].to_dict()
education_map = mappings[mappings['variable_categorica'] == 'highest_education'].set_index('valor_ordinal')[
    'valor_original'].to_dict()

# Aplicar los mapeos
df['region'] = df['region_ordinal'].map(region_map)
df['gender'] = df['gender_ordinal'].map(gender_map)
df['education'] = df['highest_education_ordinal'].map(education_map)

# Crear los gráficos
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Boxplot por región
sns.boxplot(data=df, x='region', y='predicted_score', hue='region', palette='pastel', ax=axes[0], legend=False)
axes[0].set_title('Distribución del Score por Región', fontsize=14)
axes[0].tick_params(axis='x', rotation=90)

# Violinplot por género
sns.violinplot(data=df, x='gender', y='predicted_score', hue='gender', palette='Set3', ax=axes[1], legend=False)
axes[1].set_title('Distribución del Score por Género', fontsize=14)

# Barplot por nivel educativo
mean_scores = df.groupby('education')['predicted_score'].mean().reset_index()
sns.barplot(data=mean_scores, x='education', y='predicted_score', hue='education', palette='muted', ax=axes[2],
            legend=False)
axes[2].set_title('Promedio del Score por Nivel Educativo', fontsize=14)
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(os.path.join(rutaEdaImg, "H1.2 - EDA - Variables Ordinales.png"))
plt.show()


In [ ]:
# Analisis exploratorio EDA por clustering

# Cargar los datos
df = future_df

# Definir combinaciones de variables para clustering
features = [
    ("predicted_score", "avg_credits"),
    ("predicted_score", "total_clicks"),
    ("predicted_score", "avg_clicks_per_day")
]

# Estilo visual
sns.set(style="whitegrid")
palette = sns.color_palette("husl", 3)

# Crear figura
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Generar gráficos con mejoras visuales
for i, (x_var, y_var) in enumerate(features):
    data = df[[x_var, y_var]].copy()
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)

    # KMeans clustering
    kmeans = KMeans(n_clusters=3, random_state=42)
    clusters = kmeans.fit_predict(data_scaled)
    centroids = kmeans.cluster_centers_
    centroids_original = scaler.inverse_transform(centroids)

    cluster_col = f'cluster_{i + 1}'
    df[cluster_col] = clusters

    # Gráfico de dispersión
    sns.scatterplot(
        x=x_var,
        y=y_var,
        hue=cluster_col,
        size='active_days',
        sizes=(20, 200),
        palette=palette,
        data=df,
        ax=axes[i],
        edgecolor='black',
        alpha=0.7
    )

    # Centroides
    axes[i].scatter(
        centroids_original[:, 0],
        centroids_original[:, 1],
        s=250,
        c='black',
        marker='X',
        label='Centroides'
    )

    # Anotaciones de centroides
    for j, (cx, cy) in enumerate(centroids_original):
        axes[i].annotate(f'C{j}', (cx, cy), textcoords="offset points", xytext=(0, 10), ha='center', fontsize=10,
                         color='black')

    axes[i].set_title(f'Clustering: {x_var} vs {y_var}', fontsize=14)
    axes[i].legend(title='Cluster', loc='best')

plt.tight_layout()
plt.savefig(os.path.join(rutaEdaImg, "H1.2 - Clustering y Analisis de Estudiantes por score.png"))
plt.show()

# Lista para almacenar descripciones de clusters
cluster_descriptions = []

# Aplicar clustering y generar descripciones
for i, (x_var, y_var) in enumerate(features):
    data = df[[x_var, y_var]].copy()
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)

    # KMeans clustering
    kmeans = KMeans(n_clusters=3, random_state=42)
    clusters = kmeans.fit_predict(data_scaled)
    cluster_col = f'cluster_{i + 1}'
    df[cluster_col] = clusters

    # Descripción estadística por cluster
    desc = df.groupby(cluster_col)[[x_var, y_var]].agg(['mean', 'std', 'count']).reset_index()
    desc.columns = ['cluster_id'] + [f"{col[0]} ({col[1]})" for col in desc.columns[1:]]
    desc.insert(0, 'cluster_type', cluster_col)
    cluster_descriptions.append(desc)

# Combinar todas las descripciones en un solo DataFrame
combined_description = pd.concat(cluster_descriptions, ignore_index=True)

# Mostrar la tabla combinada
display(combined_description)
combined_description.to_csv(os.path.join(rutaResult_Pred, "H1.2 - Clustering resumen perfiles clusters.csv"),
                            index=False)
